<a href="https://colab.research.google.com/github/GreenBozz/Gestionale-per-negozio/blob/main/Progetto_Gestionale_Negozio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Struttura del codice:

##Archiviazione delle informazioni:

Le informazioni sono da in un file JSON. Conterrá le informazioni sulle scorte presenti in magazzino e le informazioni di contabilitá.

##Funzioni dedicate:

####Aiuto:

Fornisce una lista degli input validi che l'utente puó inserire.

####Aggiungi:

Aggiunge un item a magazzino.csv, riportandone la quantitá, le informazioni di prezzo d'acquisto e di vendita utili per la gestione della contabilitá da parte di altre funzioni.

Se il nome Item é giá presente, chiede all'utente conferma del prezzo di vendita e di acquisto giá inseriti. Se essi sono corretti, aggiorna la numerica. Se essi non sono corretti, richiede all'utente un diverso nome per l'articolo.

Inserisce i dati della spesa d'acquisto nella contabilitá.

Infine chiede all'utente se vuole aggiungere un altro articolo.

####Elenca:

Mostra tutti gli articoli in magazzino, con il dettaglio della quantitá disponibile ed il prezzo di vendita.

####Vendita:

Richiede nome e quantitá della vendita. Verifica se in magazzino ci sono abbastanza pezzi, altrimenti fornisce un alert e chiede di reinserire il valore.

Confermato il valore, sottrae dal magazzino gli articoli venduti. Aggiungi l'introito in contabilitá.

Chiedi se si vuole vendere un'altro articolo.

####Profitti:

Fornisce lo stato della contabilitá, fornento profitto lordo e netto

####Chiudi:

Chiudi il programma.

#Struttura database

##Descrizione della Struttura del Database

La struttura del database proposta si basa su un dizionario Python con due chiavi principali: MAGAZZINO e CONTABILITÀ. La scelta di questa struttura ha come obiettivo quello di ottimizzare l'accesso, la gestione e la modifica delle informazioni, in particolare per quanto riguarda la gestione dell'inventario (MAGAZZINO), rispetto ad approcci tradizionali basati su formati come CSV o TSV.
##Dettaglio della Struttura

#####**MAGAZZINO**:
Il valore associato alla chiave MAGAZZINO è un dizionario, il cui scopo è quello di mappare ogni articolo del magazzino su una voce strutturata come segue:

**"nome_articolo":  [quantità, prezzo_acquisto_unitario,prezzo_vendita_unitario]**

Ogni articolo è quindi una chiave del dizionario, e il valore associato è una lista contenente:

* quantità (intero): la quantità disponibile dell'articolo.
* prezzo d'acquisto unitario (float): il costo di acquisto per unità dell'articolo.
* prezzo di vendita unitario (float): il prezzo a cui l'articolo viene venduto.


####**CONTABILITÀ:**
Il valore associato alla chiave CONTABILITÀ conterrà informazioni relative agli aspetti contabili del database, come registrazioni delle vendite, acquisti, bilanci, ecc. La struttura esatta del sotto-dizionario dipende dalle esigenze specifiche della contabilità aziendale.

##Vantaggi della Struttura Proposta

La struttura di database basata su un dizionario Python con due chiavi principali, MAGAZZINO e CONTABILITÀ, è preferibile rispetto a formati tradizionali come CSV e TSV per i seguenti motivi:

* **Accesso Diretto:** Il dizionario consente un accesso diretto ai singoli articoli utilizzando la chiave dell'articolo stesso. Questo elimina la necessità di effettuare iterazioni attraverso il database per cercare un articolo specifico, come invece sarebbe richiesto nei formati CSV o TSV, che comportano una ricerca basata su indice o su nome. Ad esempio, per accedere alla quantità di un articolo specifico, basta utilizzare la notazione database["magazzino"]["nome_articolo"][0], che è un'operazione in tempo costante O(1).

* **Efficienza nelle Operazioni di Modifica:** Le operazioni di aggiornamento delle informazioni, come la modifica della quantità di un articolo, sono estremamente rapide. Modificare il valore associato a una chiave (ad esempio, database["magazzino"]["nome_articolo"][0] = nuova_quantità) è un'operazione diretta e veloce, senza la necessità di cercare l'articolo come sarebbe necessario in una struttura CSV o TSV.

* **Struttura Chiara e Flessibile:** La struttura del dizionario permette una rappresentazione chiara e ben organizzata delle informazioni, dove ogni articolo è una chiave univoca con un valore strutturato (una lista con la quantità, il prezzo d'acquisto e il prezzo di vendita). Questo facilita la manutenzione del database e l'espansione delle informazioni memorizzate in futuro, se necessario.

* **Rappresentazione Semantica:** La struttura del dizionario è auto-descrittiva: la chiave MAGAZZINO è chiaramente legata alla gestione dell'inventario e le chiavi all'interno di essa rappresentano gli articoli, il che semplifica la comprensione del database anche per chi non ha familiarità con il codice. Al contrario, nei formati CSV o TSV, le informazioni devono essere interpretate in base alla posizione delle colonne, il che rende la struttura meno esplicita e meno leggibile.



In [47]:
import json

In [26]:
'''
Questa cella contiene l'inizializzazione del database, organizzata come un
dizionario e descritta nel dettaglio sopra
'''

database={
    "magazzino": {
        #articoli nel formato chiave: [ quantitá, costo singolo prodotto, prezzo singolo prodotto ]
    },

    "contabilitá":{
        "lordo": 0,
        "netto": 0
    }
}

with open ("database.json") as file_dati:
  file_dati.write(database)

In [25]:
def aggiungi (database):
  '''
  Questa funzione permette di aggiungere o aggiornare articoli nel magazzino.

  Accetta come input un dizionario con la seguente struttura:
  Database = {
      "magazzino":{
          "nome_articolo": [quantità, prezzo_acquisto, prezzo_vendita]
      },
      "contabilitá": {
          "lordo": 0,
          "netto": 0
      }
  }
  La funzione segue questo flusso operativo:
  1. Chiede all'utente di inserire il nome dell'articolo.
  2. Verifica se l'articolo è già presente nel magazzino.
    - Se l'articolo esiste:
      - Chiede le informazioni necessarie per aggiornare quantità e prezzi.
    - Se l'articolo non esiste:
      - Aggiunge una nuova voce al magazzino con le informazioni fornite dall'utente.
  3. Aggiorna i dati del magazzino nel dizionario `Database`.
  4. Aggiunge le spese d'acquisto totali in netto come valore negativo.
  '''

  articolo = str.lower(input("Inserire nome dell'articolo: "))

  if articolo in database["magazzino"].keys():
    quantitá, costo, prezzo = database["magazzino"][articolo]
    print(f"Articolo presente in magazzino. \
    \nQuantitá: {quantitá}\
    \nPrezzo di acquisto: ${costo}\
    \nPrezzo di vendita: ${prezzo}")
    try:
      aggiunta = int(input("Quantitá da aggiungere: "))
      database["magazzino"][articolo][0] += aggiunta
      print(f"Aggiunto {articolo} x {aggiunta}")
    except ValueError:
      print("Valore non valido")

  else:
    try:
      quantitá = int(input("Quantitá:"))
      costo = round(float(input("Prezzo di acquisto: $")),2)
      prezzo = round(float(input("Prezzo di vendita: $")),2)
      database["magazzino"][articolo] = [quantitá, costo, prezzo]
      print(f"Aggiunto {articolo} x {quantitá}")
    except ValueError:
      print("Valore non valido")

  database["contabilitá"]["netto"] -= costo*quantitá


  return database


In [42]:
def elenca (database):
  '''
  Questa funzione permette la visualizzazione di tutte le voci presenti
  in magazzino.
  La funzione opera iterando sulle chiavi presenti in database["magazzino"].
  Per ogniuna viene stampato nome articolo, quantitá e prezzo di vendita.
  '''

  lista_articoli=database["magazzino"].keys()

  if not len(lista_articoli):
    print("Magazzino vuoto")
  else:
    print("%-15s%-15s%-15s" % ("ARTICOLO", "QUANTITÁ", "PREZZO") )
    for articolo in lista_articoli:
      quantitá, costo, prezzo = database["magazzino"][articolo]
      print("%-15s%-15i%-15.2f" % (articolo, quantitá, prezzo))

  return



In [36]:
def main ():
  '''
  Ramo principale del programma, progettato per caricare le informazioni salvate
  e consentire all'utente di accedere a diverse funzionalità.

  Input accettati:

    aiuto: Mostra l'elenco completo dei comandi disponibili e la loro descrizione.
    aggiungi: Consente di aggiungere nuovi articoli al magazzino o aggiornare quelli esistenti.
    elenca: Visualizza il contenuto attuale del magazzino, con dettagli sugli articoli.
    vendita: Registra la vendita di uno o più articoli, aggiornando quantità e contabilità.
    profitti: Mostra un riepilogo dei profitti, suddiviso in lordo e netto.
    chiudi: Termina il programma, salvando eventuali modifiche apportate al magazzino e alla contabilità.
  '''

  with open(database.json, "w+") as file_dati:
    database = file_dati.

  input_validi = ["aiuto", "aggiungi", "elenca", "vendita", "profitti", "chiudi"]

  trigger = 1

  while trigger:

    comando = str.lower(input("Inserire comando (aiuto per lista comandi validi):"))

    if comando not in input_validi:
      print("Comando non valido")

    elif comando == "chiudi":
      trigger = 0

    elif comando == "aggiungi":
      aggiungi(database)

    elif comando == "elenca":
      elenca (database)

    elif comando == "vendita":
      pass

    elif comando == "profitti":
      pass


In [43]:
main()

Inserire comando (aiuto per lista comandi validi):elenca
ARTICOLO       QUANTITÁ       PREZZO         
seitan         20             3.20           
pane           30             1.20           
mele           12             0.60           
Inserire comando (aiuto per lista comandi validi):chiudi


In [30]:
database["contabilitá"]

{'lordo': 0, 'netto': -66.4}

In [49]:
with open ("database.json", "w+") as file_dati:
  database_str = json.dumps(database)
  file_dati.write(database_str)